# Generate esg data using dedicated data service - Python

### Overview
ESG data is a set of scores designed to objectively measure a company’s relative performance to Environment, Social and Corporate Governance. This score is defined from publicly-reported data relative to 10 main themes (emissions, environmental product innovation, human rights, shareholders, etc.).<br>

This sample demonstrates how to directly request *`sustainability data`* endpoint to retrieve detailled ESG data for a given instrument.<br>

### Inputs/outputs
Sustainability sample requires instrument's identifier as per input. It returns the sustainability data organized in several levels.

### Services used
This sample uses *gRPC requests* in order to retrieve esg information from the dedicated hosted service. The queried endpoint in this script are:
* *SustainabilityService*: to directly retrieve the data from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.sustainability.v1*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *mplfinance* as per display package

***

# Run ESG sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas mplfinance plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.sustainability.v1.sustainability_pb2 as sustainability
import systemathics.apis.services.sustainability.v1.sustainability_pb2_grpc as sustainability_service
import systemathics.helpers.token_helpers as token_helpers
import systemathics.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *sustainability* service, we need to specify an instrument identifier:
* ticker
* exchange
* source [Optionnal]

In [ ]:
# set the instrument
ticker = 'ATVI'
exchange = 'XNGS'

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# create the sustainability request
request = sustainability.SustainabilityRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the daily prices service
        service = sustainability_service.SustainabilityServiceStub(channel)
        
        # process the request
        response = service.Sustainability(
            request = request, 
            metadata = [('authorization', token)]
        )
        
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

### Step 4: Visualize data

#### 4.1 Retrieve esg data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# Helper method to define colors for level
def get_color(parent):
    if parent == '3': #Controversy
        return "red"
    elif parent == '4': #Environment
        return "orange"
    elif parent == '5': #Social
        return "purple"
    elif parent == '6': #Governance
        return "green"
    else:
        return ""

In [ ]:
#pPrepare the data frame content
values = [d.value for d in response.data]
names = [d.name for d in response.data]
ids = [d.id for d in response.data]
parents = [d.parent for d in response.data]

colors = [get_color(d.parent) for d in response.data]

d = {'Names': names, 'Values': values, 'Id': ids, 'Parents' : parents, 'Colors' : colors }
# create pandas dataframe
df = pd.DataFrame(data=d)
df = df.sort_values(by='Parents')
df

#### 4.2 Plot esg data in a circular barplot
The following code snippets display esg data in a circular barplot diagram using *`matplotlib`* open source package.

In [ ]:
def get_label_rotation(angle, offset):
    # Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle + offset)
    if angle <= np.pi:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"
    return rotation, alignment

In [ ]:
def add_labels(angles, values, labels, offset, ax):
    
    # This is the space between the end of the bar and the label
    padding = 4
    
    # Iterate over angles, values, and labels, to add all of them.
    for angle, value, label, in zip(angles, values, labels):
        angle = angle
        
        # Obtain text rotation and alignment
        rotation, alignment = get_label_rotation(angle, offset)

        # And finally add the text
        ax.text(
            x=angle, 
            y=value + padding, 
            s=label, 
            ha=alignment, 
            va="center", 
            rotation=rotation, 
            rotation_mode="anchor"
        ) 

In [ ]:
# Filter groups: we choose only the most detailled level
df_filtered = df[df['Parents'].isin(['3', '4','5','6'])]
df_filtered

In [ ]:
ANGLES = np.linspace(0, 2 * np.pi, len(df_filtered), endpoint=False)
VALUES = df_filtered["Values"].values
LABELS = df_filtered["Names"].values
COLORS = df_filtered["Colors"].values

# Determine the width of each bar. 
# The circumference is '2 * pi', so we divide that total width over the number of bars.
WIDTH = 2 * np.pi / len(VALUES)

# Determines where to place the first bar. 
# By default, matplotlib starts at 0 (the first bar is horizontal)
# but here we say we want to start at pi/2 (90 deg)
OFFSET = np.pi / 2

# Initialize Figure and Axis
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw={"projection": "polar"})

# Specify offset
ax.set_theta_offset(OFFSET)

# Set limits for radial (y) axis. The negative lower bound creates the whole in the middle.
ax.set_ylim(-100, 100)

# Remove all spines
ax.set_frame_on(False)

# Remove grid and tick marks
ax.xaxis.grid(False)
ax.yaxis.grid(False)
ax.set_xticks([])
ax.set_yticks([])

ax.set_title('ESG data for {0}-{1}'.format(ticker,exchange), size = 20)

# Add bars
ax.bar(
    ANGLES, VALUES, width=WIDTH, color = COLORS, linewidth=2, edgecolor="white"
)

# Add labels
add_labels(ANGLES, VALUES, LABELS, OFFSET, ax)

#### 4.3 Plot esg data in a kriviat diagram
The following code snippets display esg data in a kriviat diagram using *`matplotlib`* open source package.

In [ ]:
# As in previous examples, we work only with the subcategories
names = df_filtered['Names'].tolist()
values = df_filtered['Values'].tolist()

In [ ]:
# number of variable
N = len(names)

# We are going to plot the first line of the data frame.
# But we need to repeat the first value to close the circular graph:
my_plot_values = []
my_plot_values += values
my_plot_values += values[:1]

# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
my_plot_angles = [n / float(N) * 2 * np.pi for n in range(N)]
my_plot_angles += my_plot_angles[:1]

In [ ]:
# Initialise the spider plot
plt.figure(figsize=(25, 10))
ax = plt.subplot(111, polar=True)
 
# Draw one axe per variable + add labels
plt.xticks(my_plot_angles[:-1], names, color='black', size=10)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([20,40,60, 80], ["20","40","60","80"], color="grey", size=8)
plt.ylim(0,100)
##plt.yticks([10,20,30], ["10","20","30"], color="grey", size=7)
##plt.ylim(0,40)
 
# Plot data
ax.plot(my_plot_angles, my_plot_values, linewidth=1, linestyle='solid')
 
# Fill area
ax.fill(my_plot_angles, my_plot_values, 'b', alpha=0.1)

# Show the graph
plt.title('ESG data for {0}-{1}'.format(ticker,exchange), size = 25)
plt.show()